In [1]:
import pandas as pd
import numpy as np

#nilearn imports
from nilearn import plotting, image
from nilearn.image import mean_img
from nilearn.plotting import plot_anat, plot_img, plot_stat_map, show, plot_design_matrix
from nilearn.glm import threshold_stats_img
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
from nilearn.reporting import get_clusters_table

#sklearn imports
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier

In [3]:
#function to get first level weights for the runs
#possible task types include colorWheel or sameDifferent 

def extract_beta_weights(subject_id = None, task_type = 'colorWheel',  n_runs=1):
    #parameters
    tr = 2.0 
    hrf_model = "spm + derivative"
    smoothing_fwhm = 0.6
    drift_model = "cosine"
    run_betas = {
        'colorWheel': {},
        'sameDifferent': {}
    }

    for num_run in range(n_runs):
        #load subject nii files
        run_img = image.load_img(f'/Volumes/kiyonaga/MIPS/fMRI/subjects/MIPS_{subject_id}/func/run0{num_run+1}/srraf**.nii')

        #load the event file for the run
        events = pd.read_table(f'/Volumes/kiyonaga/MIPS/BIDS/sub-{subject_id}/func/sub-{subject_id}_task-{task_type.lower()}_acq-norm_run-{num_run+1}_events.tsv')

        #run the first level model
        fmri_glm = FirstLevelModel(
            t_r=tr,
            hrf_model=hrf_model,
            smoothing_fwhm=smoothing_fwhm,
            drift_model=drift_model,
            minimize_memory=False   
        )

        fmri_glm = fmri_glm.fit(run_img, events)

        #create design matrix
        design_matrix = fmri_glm.design_matrices_[0]

        #
        n_regressors = design_matrix.shape[1]
        activation = np.zeros(n_regressors)
        activation[0] = 1

        #should we remove confound with high_variance_confounds?

        # t-statistic to z-scale 
        z_map = fmri_glm.compute_contrast(activation, output_type="z_score")
        run_betas[task_type][f'run_{num_run+1}'] = {z_map}
    return run_betas

extract_beta_weights(103, 'sameDifferent')['sameDifferent']['run_1']

/Users/jennifer/anaconda3/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:129: UserWarning: 'trial_type' column not found in the given events data.
  warnings.warn(
/Users/jennifer/anaconda3/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: centralJitter, SeqStartTime, probert, probeACC, response, SetSize, ProbeLoc, chooseSequence, LastITI, ThisITI, CorrectProbeColor, error, correctResp, ProbeMatch, probeStart, block, ProbeType, ChosenProbeColor, runStart, CurrentTask, msecProbeRT, subject, counterbalance, scannerStart, stimStart, stimFlip, trialStart
  warnings.warn(


{<nibabel.nifti1.Nifti1Image at 0x178d16c50>}

In [2]:
def split_train_test_valid(X, y): #split data into 20% test 20% validation 60% training 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) #0.25 * 0.8 = 0.2
    return X_train, X_val, X_test, y_train, y_val, y_test 

In [ ]:
def train_model(X_train, y_train): 
    # SVM classifier, change depending on what model we want to test
    svm_model = SVC(kernel='linear')
    svm_model.fit(X_train, y_train)

    return svm_model

In [3]:
def test_model(model, y_test, X_test):
    # Evaluate performance
    y_pred = svm_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Model Accuracy: " + str(accuracy))

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    show()


In [ ]:
# general procedure goes as follows: make several models of varying kinds, test it using validation set
# then test the best performing model using the test set 

In [ ]:
#use to compare different models
#can also tweak this code a little bit to also test hyperparameters

def test_diff_models(X_train, y_train, X_val, y_val):    
    models = [SVC(kernel='linear'), DecisionTreeClassifier(random_state=0)] #just example models to use, can change later
    for model in models:
        model.fit(X_train, y_train)
        test_model(model,y_val, X_val)

